# **Upload coverted OPT model to huggingface**
This is the way I found to deal with converting to gqa and then load again with custom loader in order for the BitsAndBytesConfig to work.
This is also usfull for saving time by not saving locally and reloading.

In [ ]:
! pip install -q transformers==4.52.2
! pip -q install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 124.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

import sys
sys.path.append('/content/drive/MyDrive/Transformers/Project_OPT')

from OPT_to_GQA import convert_opt_to_gqa

In [ ]:
from huggingface_hub import login

login()

In [ ]:
def create_standard_heads_grouping_arr(num_layers: int, num_heads: int, kv_heads: int):
    assert num_heads % kv_heads == 0, "num_heads must be divisible by kv_heads"
    group_size = num_heads // kv_heads
    grouping = [i for i in range(kv_heads) for _ in range(group_size)]
    return [grouping.copy() for _ in range(num_layers)]


heads_grouping_arr = [[13, 14, 4, 12, 15, 13, 6, 14, 6, 10, 3, 7, 1, 4, 15, 11, 9, 7, 2, 0, 0, 8, 5, 9, 3, 1, 2, 11, 5, 12, 10, 8], [3, 9, 0, 12, 8, 4, 15, 0, 7, 2, 6, 5, 13, 11, 13, 7, 4, 12, 5, 10, 14, 15, 11, 1, 3, 9, 10, 6, 8, 2, 14, 1], [3, 5, 4, 11, 6, 12, 9, 10, 2, 7, 4, 0, 3, 6, 7, 1, 8, 14, 8, 9, 12, 2, 10, 13, 13, 15, 15, 0, 1, 14, 5, 11], [9, 10, 14, 5, 7, 12, 15, 11, 8, 8, 11, 2, 0, 9, 4, 6, 12, 2, 3, 7, 14, 13, 6, 15, 10, 1, 0, 1, 4, 5, 13, 3], [8, 3, 0, 14, 12, 10, 4, 4, 13, 5, 12, 8, 14, 2, 9, 0, 10, 11, 9, 15, 1, 7, 2, 11, 6, 15, 3, 7, 6, 1, 5, 13], [14, 5, 3, 13, 2, 13, 15, 8, 9, 11, 10, 3, 0, 6, 10, 1, 8, 9, 0, 14, 7, 7, 4, 1, 11, 2, 6, 12, 5, 12, 4, 15], [14, 8, 11, 14, 9, 13, 8, 12, 5, 3, 2, 12, 11, 10, 10, 3, 13, 2, 0, 1, 6, 1, 0, 4, 7, 9, 15, 7, 15, 5, 4, 6], [9, 4, 13, 3, 6, 12, 11, 9, 5, 14, 1, 5, 7, 15, 4, 8, 13, 14, 3, 15, 10, 10, 6, 2, 7, 0, 2, 1, 8, 11, 0, 12], [10, 6, 2, 5, 9, 8, 13, 3, 1, 7, 12, 11, 0, 14, 1, 5, 15, 12, 4, 9, 0, 13, 10, 8, 4, 3, 14, 15, 2, 7, 11, 6], [6, 14, 6, 9, 7, 15, 1, 3, 8, 11, 4, 0, 13, 10, 8, 15, 5, 0, 2, 2, 7, 14, 12, 5, 4, 9, 11, 13, 1, 10, 3, 12], [6, 3, 12, 7, 3, 15, 0, 8, 12, 10, 2, 2, 9, 14, 14, 8, 13, 5, 5, 9, 10, 15, 6, 0, 7, 1, 1, 11, 11, 4, 13, 4], [5, 4, 12, 2, 6, 12, 0, 10, 6, 2, 14, 9, 7, 9, 14, 8, 5, 1, 8, 13, 1, 15, 11, 3, 7, 3, 10, 11, 13, 0, 15, 4], [3, 7, 8, 10, 12, 13, 5, 13, 3, 14, 10, 14, 15, 15, 4, 6, 11, 9, 8, 5, 0, 7, 2, 9, 12, 6, 0, 4, 11, 1, 2, 1], [4, 11, 4, 7, 0, 7, 0, 12, 6, 14, 9, 13, 2, 9, 11, 2, 8, 10, 15, 5, 1, 3, 1, 8, 10, 3, 6, 15, 13, 5, 14, 12], [10, 3, 8, 3, 4, 13, 0, 12, 15, 14, 11, 2, 7, 2, 6, 9, 9, 5, 1, 12, 13, 5, 8, 0, 14, 10, 11, 1, 7, 15, 6, 4], [2, 11, 8, 6, 15, 8, 4, 10, 1, 5, 6, 1, 4, 3, 14, 0, 9, 13, 9, 2, 7, 0, 13, 12, 7, 11, 12, 3, 10, 15, 14, 5], [14, 9, 11, 12, 6, 8, 10, 15, 12, 11, 4, 2, 7, 13, 10, 6, 2, 0, 0, 14, 8, 1, 3, 5, 4, 9, 13, 5, 1, 7, 15, 3], [14, 0, 11, 4, 1, 8, 3, 5, 8, 14, 7, 15, 7, 15, 12, 1, 9, 2, 9, 6, 6, 5, 3, 12, 13, 4, 0, 10, 13, 11, 2, 10], [5, 3, 2, 2, 7, 3, 0, 11, 13, 8, 6, 9, 14, 13, 15, 14, 4, 11, 9, 4, 12, 7, 1, 10, 1, 10, 12, 5, 8, 6, 0, 15], [12, 4, 5, 1, 2, 8, 15, 13, 2, 11, 3, 7, 5, 1, 14, 9, 11, 9, 15, 10, 6, 0, 3, 10, 14, 0, 7, 4, 8, 13, 6, 12], [3, 9, 0, 8, 12, 14, 10, 2, 4, 8, 3, 2, 9, 15, 4, 10, 15, 7, 6, 14, 7, 6, 11, 5, 13, 5, 11, 13, 0, 12, 1, 1], [10, 3, 11, 7, 13, 1, 0, 3, 4, 14, 6, 11, 9, 6, 2, 0, 15, 5, 2, 13, 15, 8, 10, 8, 5, 1, 12, 9, 14, 12, 7, 4], [3, 3, 1, 2, 7, 5, 11, 12, 12, 13, 4, 7, 0, 9, 10, 0, 8, 6, 14, 9, 6, 13, 14, 15, 4, 5, 15, 1, 10, 8, 11, 2], [11, 4, 2, 14, 10, 3, 13, 7, 15, 0, 7, 1, 8, 2, 9, 4, 15, 13, 11, 5, 5, 0, 1, 9, 12, 10, 8, 6, 12, 6, 3, 14]]


In [ ]:
repo_size = "1.3b"
repo_id = f"facebook/opt-{repo_size}"
model = AutoModelForSequenceClassification.from_pretrained(
   repo_id, num_labels=3, device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-1.3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
kv_heads = 16
num_heads = 32
group_size = num_heads // kv_heads
num_layers = len(model.model.decoder.layers)
# heads_grouping_arr = create_standard_heads_grouping_arr(num_layers, num_heads,
#                                                         kv_heads)

model_gqa = convert_opt_to_gqa(model, kv_heads=kv_heads, heads_grouping_arr=heads_grouping_arr, inplace=True)
print(f"Converted to GQA with {heads_grouping_arr}")
model_gqa

Converted to GQA with [[13, 14, 4, 12, 15, 13, 6, 14, 6, 10, 3, 7, 1, 4, 15, 11, 9, 7, 2, 0, 0, 8, 5, 9, 3, 1, 2, 11, 5, 12, 10, 8], [3, 9, 0, 12, 8, 4, 15, 0, 7, 2, 6, 5, 13, 11, 13, 7, 4, 12, 5, 10, 14, 15, 11, 1, 3, 9, 10, 6, 8, 2, 14, 1], [3, 5, 4, 11, 6, 12, 9, 10, 2, 7, 4, 0, 3, 6, 7, 1, 8, 14, 8, 9, 12, 2, 10, 13, 13, 15, 15, 0, 1, 14, 5, 11], [9, 10, 14, 5, 7, 12, 15, 11, 8, 8, 11, 2, 0, 9, 4, 6, 12, 2, 3, 7, 14, 13, 6, 15, 10, 1, 0, 1, 4, 5, 13, 3], [8, 3, 0, 14, 12, 10, 4, 4, 13, 5, 12, 8, 14, 2, 9, 0, 10, 11, 9, 15, 1, 7, 2, 11, 6, 15, 3, 7, 6, 1, 5, 13], [14, 5, 3, 13, 2, 13, 15, 8, 9, 11, 10, 3, 0, 6, 10, 1, 8, 9, 0, 14, 7, 7, 4, 1, 11, 2, 6, 12, 5, 12, 4, 15], [14, 8, 11, 14, 9, 13, 8, 12, 5, 3, 2, 12, 11, 10, 10, 3, 13, 2, 0, 1, 6, 1, 0, 4, 7, 9, 15, 7, 15, 5, 4, 6], [9, 4, 13, 3, 6, 12, 11, 9, 5, 14, 1, 5, 7, 15, 4, 8, 13, 14, 3, 15, 10, 10, 6, 2, 7, 0, 2, 1, 8, 11, 0, 12], [10, 6, 2, 5, 9, 8, 13, 3, 1, 7, 12, 11, 0, 14, 1, 5, 15, 12, 4, 9, 0, 13, 10, 8, 4, 3, 14, 15, 2

OPTForSequenceClassification(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTGQAAttention(
            (k_proj): Linear(in_features=2048, out_features=1024, bias=True)
            (v_proj): Linear(in_features=2048, out_features=1024, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_la

In [ ]:
saved_path = f"facebook-opt-{repo_size}-GQA-{kv_heads}-kv-input-qclusterd"
model_gqa.push_to_hub(repo_id=saved_path)

Uploading...:   0%|          | 0.00/4.86G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/joshwapanda/facebook-opt-1.3b-GQA-16-kv-input-qclusterd/commit/acd785bb67497a21a85dd77405294da1812f7db9', commit_message='Upload OPTForSequenceClassification', commit_description='', oid='acd785bb67497a21a85dd77405294da1812f7db9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/joshwapanda/facebook-opt-1.3b-GQA-16-kv-input-qclusterd', endpoint='https://huggingface.co', repo_type='model', repo_id='joshwapanda/facebook-opt-1.3b-GQA-16-kv-input-qclusterd'), pr_revision=None, pr_num=None)